In [1]:
from pathlib import Path
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, differential_evolution, basinhopping
from pandas import Series, DataFrame
import pandas as pd
import teslamax
from teslamax import TeslaMaxGeometry, TeslaMaxPreDesign, TeslaMaxModel

In [2]:
TABLE_FILE = Path('analytical-fitting-points.csv')
RESULTS_KAPPA_FILE_PREFIX = 'results_kappa'

In [3]:
table = pd.read_csv(TABLE_FILE,index_col=0)

In [4]:
nrows, ncolumns = table.shape

In [5]:
FIXED_PARAMETERS = {
        "h_fc": 0.025,
        "R_e": 1,
        "n_II": 5,
        "n_IV": 5,
        "phi_C_II": 0,
        "phi_S_II": 60,
        "phi_S_IV": 60,
        "mu_r_II": 1.05,
        "mu_r_IV": 1.05,
        "linear_iron": 1,
        "mu_r_iron": 5e2,
        "l_element_min": 1e-3,
        "l_element_max": 3e-2,
        "n_narrow": 2,
             }

n_II = FIXED_PARAMETERS["n_II"]
n_IV = FIXED_PARAMETERS["n_IV"]

n = n_II + n_IV

B_rem = 1.26

# expand parameters to include remanence magnitudes for each segment
FIXED_PARAMETERS = teslamax.expand_parameters_from_remanence_array(B_rem*np.ones(n), 
                                                        FIXED_PARAMETERS, 
                                                        "B_rem")

In [6]:
parameters_list = []
B_est_list = []
for isimulation, row in table.iterrows():
    parameters = FIXED_PARAMETERS.copy()
    parameters["R_i"] = 1e-3*row["R1  (Ri)"]
    parameters["R_o"] = 1e-3*row["R2  (Ro)"]
    parameters["R_g"] = 1e-3*row["R3  (Rg)"]
    parameters["R_s"] = 1e-3*row["R4  (Rs)"]
    
    parameters_list.append(parameters.copy())
    
    B_est_list.append(row["B_estimado"])

In [7]:
B_min = 0.5
B_max = 1.5
n_B_values = 3
B_values = np.linspace(B_min,B_max,n_B_values)

In [8]:
Kappa_data = np.zeros((nrows,n_B_values))

In [9]:
target_function = teslamax.calculate_ramp_profile
B_low = 0.0
field_fraction = 0.35
    
for i in 0,:
    
    results_path = Path(RESULTS_KAPPA_FILE_PREFIX + "_%s.txt" %(i,))
    results_path.write_text("B[T]\tKappa[]\n")
    
    tmpd = TeslaMaxPreDesign(parameters_list[i])
    
    for j, B_high in enumerate(B_values):
        
        target_args = (B_high,B_low,field_fraction)
        
        
        alpha_B_rem_g = tmpd.get_optimal_remanence_angles(target_function,target_args)
        Kappa = tmpd.calculate_functional_target(alpha_B_rem_g,
                                                    target_function,
                                                    target_args)
        Kappa_data[i,j] = Kappa
            
        results_str = "%.2f\t%.4f\n" %(B_high,Kappa,)
        with results_path.open(mode='a',buffering=1) as f:
                f.write(results_str)
                f.flush()
                os.fsync(f.fileno())

In [10]:
Kappa_data

array([[0.04970835, 0.11908451, 0.20033356],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])